In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K

from tensorflow.keras import regularizers
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, Concatenate, BatchNormalization, Average, Convolution2D, MaxPooling2D, Activation, GlobalMaxPooling2D, AveragePooling2D, GlobalAveragePooling2D 
from tensorflow.keras.models import Model, Sequential
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import plot_model, to_categorical, Sequence
from tensorflow.keras import initializers
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2

from sklearn.metrics import classification_report
from itertools import tee

import numpy as np
import matplotlib.pyplot as plt

print("tensorflow:", tf.__version__)
print("tf.keras:", tf.keras.__version__)
# print(tf.python.keras.preprocessing.__version__)

tensorflow: 2.0.1
tf.keras: 2.2.4-tf


In [2]:
# ssss

In [3]:
if tf.__version__.startswith("1."):
    val_acc_name = "val_acc"
    acc_name = "acc"
else:
    tf.compat.v1.disable_eager_execution()
    val_acc_name = "val_accuracy"
    acc_name = "accuracy"

In [4]:
K.image_data_format()

'channels_last'

In [5]:
class_names = ["cat", "dog"]
batch_size = 128 # 32 # 64 # 128
image_shape = (299, 299, 3)

In [6]:
X_train = np.load("X_train.npy")
X_valid = np.load("X_valid.npy")
X_test = np.load("X_test.npy")

y_train = np.load("y_train.npy")
y_valid = np.load("y_valid.npy")
y_test = np.load("y_test.npy")

In [7]:
# X_train = X_train / 255.
# X_valid = X_valid / 255.
# X_test = X_test / 255.

In [8]:
# X_train = X_train[:1000]
# X_valid = X_valid[:500]
# X_test = X_test[:300]

# y_train = y_train[:1000]
# y_valid = y_valid[:500]
# y_test = y_test[:300]

In [9]:
print("X_train.shape:", str(X_train.shape))
print("X_valid.shape:", str(X_valid.shape))
print("X_test.shape:", str(X_test.shape))
print()
print("y_train.shape:", str(y_train.shape))
print("y_valid.shape:", str(y_valid.shape))
print("y_test.shape:", str(y_test.shape))

X_train.shape: (17462, 299, 299, 3)
X_valid.shape: (3742, 299, 299, 3)
X_test.shape: (3742, 299, 299, 3)

y_train.shape: (17462, 1)
y_valid.shape: (3742, 1)
y_test.shape: (3742, 1)


In [10]:
y_train = to_categorical(y_train, len(class_names))
y_valid = to_categorical(y_valid, len(class_names))
y_test = to_categorical(y_test, len(class_names))

In [11]:
print("y_train.shape:", str(y_train.shape))
print("y_valid.shape:", str(y_valid.shape))
print("y_test.shape:", str(y_test.shape))

y_train.shape: (17462, 2)
y_valid.shape: (3742, 2)
y_test.shape: (3742, 2)


In [12]:
%%time

datagen_aug = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.15,
    height_shift_range=0.15,
    horizontal_flip=True,
    zoom_range=0.15,
    rescale=1./255
    )
datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen_aug.flow(X_train, y_train, batch_size=batch_size)
valid_generator = datagen.flow(X_valid, y_valid, batch_size=batch_size)
test_generator = datagen.flow(X_test, y_test, batch_size=batch_size, shuffle=False)


CPU times: user 2.31 s, sys: 46.7 s, total: 49 s
Wall time: 49 s


In [13]:
def make_model():
    inputs = Input(shape=image_shape, name="Input")
    base_model = InceptionResNetV2(include_top=False, 
                               input_shape=image_shape, 
                               input_tensor=inputs, 
                               weights='imagenet', 
                               classes=2)
    base_model.trainable = False
    batch_norm1 = BatchNormalization()(base_model.output)
    flat1 = Flatten()(batch_norm1)
    dense1 = Dense(100, activation="relu")(flat1)
    dense2 = Dense(len(class_names), activation="softmax")(dense1)

    model = Model(inputs=[inputs], outputs=[dense2])
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

In [14]:
%%time
K.clear_session()

model = make_model()
# model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
CPU times: user 33.3 s, sys: 2.96 s, total: 36.3 s
Wall time: 35.3 s


In [15]:
# plot_model(model, to_file='model.png')

In [16]:
checkpoint = ModelCheckpoint('cat_dogs_with_InceptionResNetV2.hdf5',
                            monitor=val_acc_name,
                            save_best_only=True,
                            period=2,
                            verbose=1)
early_stopping_callback = EarlyStopping(monitor=val_acc_name, 
                                        min_delta=1e-6, 
                                        patience=50, 
                                        verbose=1, 
                                        mode='auto', 
                                        baseline=None, 
                                        restore_best_weights=True)
learning_rate_reduction = ReduceLROnPlateau(monitor=val_acc_name,
                                           patience=5,
                                           verbose=1,
                                           factor=0.5,
                                           min_lr=0.00000001)

In [ ]:
%%time
            
# Обучаем модель при помощи генератора пакетов
history = model.fit_generator(train_generator, 
                              steps_per_epoch=train_generator.n//train_generator.batch_size, 
                              epochs=5, 
                              validation_data=valid_generator, 
                              validation_steps=valid_generator.n//valid_generator.batch_size,
                             callbacks=[checkpoint, learning_rate_reduction, early_stopping_callback],
                             workers=10,
                             max_queue_size=10,
                             use_multiprocessing=False)

Epoch 1/5
136/136 [==============================] - 232s 2s/step - loss: 0.2763 - accuracy: 0.9673 - val_loss: 0.3394 - val_accuracy: 0.9569
Epoch 2/5
135/136 [============================>.] - ETA: 1s - loss: 0.1084 - accuracy: 0.9795
Epoch 00002: val_accuracy improved from -inf to 0.95690, saving model to cat_dogs_with_InceptionResNetV2.hdf5
136/136 [==============================] - 550s 4s/step - loss: 0.1082 - accuracy: 0.9794 - val_loss: 0.1872 - val_accuracy: 0.9569
Epoch 3/5
136/136 [==============================] - 209s 2s/step - loss: 0.0694 - accuracy: 0.9817 - val_loss: 0.1447 - val_accuracy: 0.9784
Epoch 4/5
135/136 [============================>.] - ETA: 1s - loss: 0.0626 - accuracy: 0.9826
Epoch 00004: val_accuracy improved from 0.95690 to 0.97764, saving model to cat_dogs_with_InceptionResNetV2.hdf5
136/136 [==============================] - 216s 2s/step - loss: 0.0625 - accuracy: 0.9825 - val_loss: 0.1654 - val_accuracy: 0.9776
Epoch 5/5
 90/136 [==================>.

In [ ]:
plt.figure()
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='validation loss')
plt.xlabel('Эпоха обучения')
plt.ylabel('Значение функции потерь')
plt.legend()

plt.figure()
plt.plot(history.history[acc_name], label='train accuracy')
plt.plot(history.history[val_acc_name], label='validation accuracy')
plt.xlabel('Эпоха обучения')
plt.ylabel('Доля верных ответов')
plt.legend()

plt.show()

In [ ]:
test_generator.reset()
model.evaluate_generator(test_generator, steps=test_generator.n//test_generator.batch_size, workers=10, max_queue_size=10, verbose=1)

In [ ]:
test_steps = test_generator.n//test_generator.batch_size

test_generator.reset()
y_true = np.vstack(next(test_generator)[1] for _ in range(test_steps)).astype('int')

test_generator.reset()
y_pred = model.predict_generator(test_generator, steps=test_steps, workers=10, max_queue_size=10, verbose=1)

print("Test data evaluation:")
print(classification_report(y_true.argmax(axis=1), y_pred.argmax(axis=1), target_names=class_names))

In [ ]:
old_weights = model.get_weights()

In [ ]:
del model
K.clear_session()

model = make_model()
model.set_weights(old_weights)
model.save("cat_dogs_with_InceptionResNetV2_WITHOUT_OPTIMIZER.hdf5")

In [ ]:
test_generator.reset()
model.evaluate_generator(test_generator, steps=test_generator.n//test_generator.batch_size, workers=10, max_queue_size=10, verbose=1)

In [ ]:
test_steps = test_generator.n//test_generator.batch_size

test_generator.reset()
y_true = np.vstack(next(test_generator)[1] for _ in range(test_steps)).astype('int')

test_generator.reset()
y_pred = model.predict_generator(test_generator, steps=test_steps, workers=10, max_queue_size=10, verbose=1)

print("Test data evaluation:")
print(classification_report(y_true.argmax(axis=1), y_pred.argmax(axis=1), target_names=class_names))

In [ ]:
import os

In [ ]:
print("cat_dogs_with_InceptionResNetV2.hdf5 file size: %.3f Megabytes" % (os.stat("cat_dogs_with_InceptionResNetV2.hdf5").st_size / 1024 / 1024))

In [ ]:
print("cat_dogs_with_InceptionResNetV2_WITHOUT_OPTIMIZER.hdf5 file size: %.3f Megabytes" % (os.stat("cat_dogs_with_InceptionResNetV2_WITHOUT_OPTIMIZER.hdf5").st_size / 1024 / 1024))